In [ ]:
%matplotlib inline
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
from scipy import ndimage
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')

### Real(ish) World Example
#### Creating a active transit priority index for Oakland

Please borrow and improve on the code! <br>
It is set up to run with minimal input from the user. <br>
It uses census shapefiles, crash data from TIMS, and census demographic information.

#### Set Geography for Analysis

Start by selecting the spatial information for the Census block group. <br>
Start by Downloading Geography <br>
Files are from Census

In [ ]:
census_places = gpd.read_file("Spatial Files/tl_2019_06_place.shp")

In [ ]:
# Start by selecting Oakland from places file using name

place = census_places[census_places['NAME'] == 'Oakland']

In [ ]:
# In this code, we use the overlay function to select only for Census Block groups inside of Oakland, and create a new shapefile.
#We're not going to run this today because it takes a long time to process
#block_gr = gpd.read_file("Spatial Files/AlamedaBlocks.shp")
#bl_gr = gpd.overlay(place, block_gr, how = "intersection")
#bl_gr.to_file('OaklandBlocks.shp', driver='ESRI Shapefile')

In [ ]:
# Here we'll read in the block group file created by the cell above. GeoID_2 represents the block group code
bl_gr = gpd.read_file("Spatial Files/OaklandBlocks.shp")
bl_gr.head()

In [ ]:
# Make a simple copy of the bl_gr file that retains id and geography

bl_simp = bl_gr[['GEOID_2','geometry']].reset_index()
place_simp = place[['NAME','geometry']]

In [ ]:
# See what we have

bl_simp.plot(figsize = (5, 5), color = "whitesmoke", edgecolor = "lightgrey", linewidth = 0.5).set_axis_off()

In [ ]:
# Let's drop the ugly hanging Tract that is entirely in the bay.
# I identified it by fiding the tract with the most water in it
bl_simp = bl_simp[bl_simp['GEOID_2'] != '060019900000']

In [ ]:
bl_simp.plot(figsize = (5, 5), color = "whitesmoke", edgecolor = "lightgrey", linewidth = 0.5).set_axis_off()

#### Attach Geospatial Data - Ped and Bike Crashes
This part of the code loads and cleans crash data from TIMS. <br>
The crashes are for 5 years of data for Oakland.

In [ ]:
ped_cr = gpd.read_file("Spatial Files/oakland_ped_collisions.shp")
bike_cr = gpd.read_file("Spatial Files/oakland_bike_collisions.shp")

In [ ]:
# Create smaller simpler dataframe
# Saving Case_ID, Crash Severity, and geometry
# Crash severity is in the column labelled "COLLISIO_1

ped_cr_sh = ped_cr[['CASE_ID','COLLISIO_1','geometry']]
ped_cr_sh = ped_cr_sh.rename(columns={"CASE_ID": "ped_ID", "COLLISIO_1": "ped_sev"})
ped_cr_sh['count_ped'] = 1
bike_cr_sh = bike_cr[['CASE_ID','COLLISIO_1','geometry']]
bike_cr_sh = bike_cr_sh.rename(columns={"CASE_ID": "bike_ID", "COLLISIO_1": "bike_sev"})
bike_cr_sh['count_bike'] = 1

In [ ]:
# The block data didn't have a projection associated with it, let's add the projection used by the census
bl_simp.crs = {'init' :'epsg:4326'}
bl_simp.crs

In [ ]:
# Reproject census block group data into the same projection as the pedestrian and bike crash data
bl_proj = bl_simp.to_crs(ped_cr_sh.crs)

In [ ]:
# Do the same thing for the place shapefile
place_simp.crs = {'init' :'epsg:4326'}
place_proj = place_simp.to_crs(ped_cr_sh.crs)

In [ ]:
# Remove crashes that are outside of Oaklands borders
bike_cr_sh = gpd.sjoin(bike_cr_sh, place_proj, how = 'inner').drop(columns = ['NAME','index_right'])

In [ ]:
# Remove crashes that are outside of Oaklands borders

ped_cr_sh = gpd.sjoin(ped_cr_sh, place_proj, how = 'inner').drop(columns = ['NAME','index_right'])

In [ ]:
# Plot what we have

base = bl_proj.plot(figsize = (10, 10), color = "whitesmoke", edgecolor = "lightgrey", linewidth = 0.5)
ped_cr_sh.plot(ax =base, markersize=4, color = 'Blue')
bike_cr_sh.plot(ax =base, markersize=2, color = 'Purple')

### ACTION STEP - 1
#### Determine Distance Consideration
How close should a crash be to a block group to be considered an impact on residents?<br/>
We can use buffering to change the area under consideration. 

In [ ]:
# We can look at the CRS to determine the projection being used and the scale for measurements.
# We are using us-ft. See: http://prj2epsg.org/epsg/2227

ped_cr_sh.crs

#### ACTION STEP IS HERE
How close to a block group should be a crash be for us to consider it
an impact on them? <br>
You decide? (Remember we are measuring in feet)

In [ ]:
ped_buffer = 5280/4  # Quarter mile
bike_buffer = (5280) # Mile

In [ ]:
ped_cr_sh['geometry'] = ped_cr_sh['geometry'].buffer(ped_buffer)
bike_cr_sh['geometry'] = bike_cr_sh['geometry'].buffer(bike_buffer)

In [ ]:
# See what we created
# Circles show the "influence" areas for crashes
# Alpha - use for creating transparency

ax = bl_proj.plot(figsize = (10, 10), color = "whitesmoke", edgecolor = "lightgrey", linewidth = 0.1)
ax = bike_cr_sh.plot(ax =ax, facecolor = "none", edgecolor = "Purple", alpha = 0.1)
ped_cr_sh.plot(ax =ax, facecolor = "none", edgecolor = "Blue", alpha = 0.1).set_axis_off()

### ACTION STEP - 2
#### Weight crashes by severity
We are now going to create a metric based on the severity of the crashes. <br>
How should we compare more or less severe crashes. Not all crashes are equal <br>
but how should we compare them to create a composite score?

There are four types of severities in the dataset (note Property Damage Only crashes are not included).

##### What severity should be given to each crash

In [ ]:
# You decide... the default gives fatal and severe crashes five times as much weight.

fatal = 5
severe = 5
visible = 1
comp_pain = 1

##### You can ignore this part of the code if you would like...
It is creating the weighted score.

In [ ]:
# This part of the code adds a severity weight column

ped_cr_sh['p_sev_wt'] = 1
ped_cr_sh['p_sev_wt'][ped_cr_sh['ped_sev'] == 1] = fatal
ped_cr_sh['p_sev_wt'][ped_cr_sh['ped_sev'] == 2] = severe
ped_cr_sh['p_sev_wt'][ped_cr_sh['ped_sev'] == 3] = visible
ped_cr_sh['p_sev_wt'][ped_cr_sh['ped_sev'] == 4] = comp_pain

bike_cr_sh['b_sev_wt'] = 1
bike_cr_sh['b_sev_wt'][bike_cr_sh['bike_sev'] == 1] = fatal
bike_cr_sh['b_sev_wt'][bike_cr_sh['bike_sev'] == 2] = severe
bike_cr_sh['b_sev_wt'][bike_cr_sh['bike_sev'] == 3] = visible
bike_cr_sh['b_sev_wt'][bike_cr_sh['bike_sev'] == 4] = comp_pain

In [ ]:
# Join crashes to block groups. Sjoin will create a location record for each overlap of a crash buffer and the block group

bl_ped = gpd.sjoin(bl_proj, ped_cr_sh, how = 'left').drop(columns = ['index_right'])
bl_bike = gpd.sjoin(bl_proj, bike_cr_sh, how = 'left').drop(columns = ['index_right'])

In [ ]:
# Summarizes the data into tables organized by block group.

bl_sum_ped = bl_ped.groupby(['GEOID_2'])['count_ped','p_sev_wt'].sum().reset_index()
bl_sum_bike = bl_bike.groupby(['GEOID_2'])['count_bike','b_sev_wt'].sum().reset_index()

In [ ]:
# Joins the data into a single table.

bl_analysis = bl_sum_ped.merge(bl_sum_bike, on = 'GEOID_2')
bl_analysis = bl_simp.merge(bl_analysis, on = 'GEOID_2')

In [ ]:
bl_analysis.head()

#### Fun with Maps!!!
Python maps are not a pretty as what you can make in other programs...
BUT you can make a lot of maps really quickly. The code below creates
a map based on the information in each column in the list.
Feel free to steal the code to generate your own maps.

Note - Alex is running through a fancier map code that is available on the desktop version of the files.  Unfortunately, the shapefiles are too large for jupyterhub to handle, so we're simplifying here.  But you can still make fast maps!

In [ ]:
for x in ['count_ped', 'p_sev_wt','count_bike','p_sev_wt']:
    ax = bl_analysis.plot(figsize = (10, 10), scheme = "percentiles", k = 6, column = x, cmap = "BuPu",edgecolor = "lightgrey", linewidth = 0.5, legend = True)
    ax.set_axis_off()
    ax.set_title(x, fontsize=25)

#### Normalizing the data
For our purposes we will want to normalize the variables summarizing the crash data. <br>
This will allow us to more easily factor the results into a composite index in the next step.

In [ ]:
# Create scores normalized as percent of highest value
bl_analysis['c_ped_perc'] = bl_analysis['count_ped']/bl_analysis['count_ped'].max()
bl_analysis['sev_ped_perc'] = bl_analysis['p_sev_wt']/bl_analysis['p_sev_wt'].max()
bl_analysis['c_bike_perc'] = bl_analysis['count_bike']/bl_analysis['count_bike'].max()
bl_analysis['sev_bike_perc'] = bl_analysis['b_sev_wt']/bl_analysis['b_sev_wt'].max()

### Add Census Information
This step brings in census data that we've downloaded and done some minor 
cleaning to.

In [ ]:
indices = pd.read_csv("alamedacounty_mostly_computed_indices.csv").fillna(0)

In [ ]:
indices.columns

##### Join the table to our work from above based on the block group
We've also included some work to drop columns that are not necessary to keep things more clean.

In [ ]:
bl_analysis['join_id'] = bl_analysis['GEOID_2']

In [ ]:
indices['join_id'] = indices['Geo_GEOID'].str[7:]

In [ ]:
bl_analysis = bl_analysis.merge(indices, on = 'join_id').drop(columns = ['join_id', 'Geo_FIPS', 'Geo_GEOID', 'Geo_NAME', 'Geo_QName',
       'Geo_STUSAB', 'Geo_SUMLEV', 'Geo_GEOCOMP', 'Geo_FILEID', 'Geo_LOGRECNO',
       'Geo_US', 'Geo_REGION', 'Geo_DIVISION', 'Geo_STATECE', 'Geo_STATE',
       'Geo_COUNTY', 'Geo_COUSUB'])

##### Let's look at what we have!
We are going to use that same mapping code from before to see what the variables look like.

In [ ]:
bl_analysis.columns

In [ ]:
for x in ['avg.hh.size', 'coc.population', 'total.population',
       'elderly.population', 'households', 'limited.english.households',
       'poverty.population', 'pop.poverty.determined', 'single.parent.fam',
       'families', 'youth.population', 'occupied.housing.units',
       'zero.vehicle.hh', 'coc.population.pct', 'elderly.population.pct',
       'limited.english.households.pct', 'poverty.population.pct',
       'single.parent.fam.pct', 'youth.population.pct', 'zero.vehicle.hh.pct',
       'coc.pop.norm', 'elderly.pop.norm', 'limited.english.households.norm',
       'poverty.population.norm', 'single.parent.fam.norm',
       'youth.population.norm', 'zero.vehicle.hh.norm']:
    ax = bl_analysis.plot(figsize = (10, 10), scheme = "percentiles", k = 6, column = x, cmap = "BuPu",edgecolor = "lightgrey", linewidth = 0.5, legend = True)
    ax.set_axis_off()
    ax.set_title(x, fontsize=25)

### ACTION STEP - 3
#### Where should we prioritize active transit investments?
We have all the data. But now the hard questions... <br>
What factors should we consider to prioritize resources? <br>
We included three starter indices: (1) Crash Only, (2) Census Factors Only, (3) Simple All Factors.
They are not that good. Make us something better... whatever that means. <br>
Go down to indices 4 and 5 to start making your own.

In [ ]:
bl_analysis['index1'] = \
        bl_analysis['c_ped_perc'] * 1 + \
        bl_analysis['sev_ped_perc'] * 1 + \
        bl_analysis['c_bike_perc'] * 1 + \
        bl_analysis['sev_bike_perc'] * 1

In [ ]:
bl_analysis['index2'] = \
        bl_analysis['coc.population.pct'] * 1 + \
        bl_analysis['elderly.population.pct'] * 1 + \
        bl_analysis['limited.english.households.pct'] * 1 + \
        bl_analysis['poverty.population.pct'] * 1 + \
        bl_analysis['single.parent.fam.pct'] * 1 + \
        bl_analysis['youth.population.pct'] * 1 + \
        bl_analysis['zero.vehicle.hh.pct'] * 1 + \
        bl_analysis['coc.pop.norm'] * 1 + \
        bl_analysis['elderly.pop.norm'] * 1 + \
        bl_analysis['limited.english.households.norm'] * 1 + \
        bl_analysis['poverty.population.norm'] * 1 + \
        bl_analysis['single.parent.fam.norm'] * 1 + \
        bl_analysis['youth.population.norm'] * 1 + \
        bl_analysis['zero.vehicle.hh.norm'] * 1

In [ ]:
bl_analysis['index3'] = \
        bl_analysis['coc.population.pct'] * 1 + \
        bl_analysis['elderly.population.pct'] * 1  + \
        bl_analysis['limited.english.households.pct'] * 1 + \
        bl_analysis['poverty.population.pct'] * 1 + \
        bl_analysis['single.parent.fam.pct'] * 1 + \
        bl_analysis['youth.population.pct'] * 1 + \
        bl_analysis['zero.vehicle.hh.pct'] * 1 + \
        bl_analysis['coc.pop.norm'] * 1  + \
        bl_analysis['elderly.pop.norm'] * 1 + \
        bl_analysis['limited.english.households.norm'] * 1 + \
        bl_analysis['poverty.population.norm'] * 1 + \
        bl_analysis['single.parent.fam.norm'] * 1 + \
        bl_analysis['youth.population.norm'] * 1 + \
        bl_analysis['zero.vehicle.hh.norm'] * 4 + \
        bl_analysis['count_ped']/bl_analysis['count_ped'].max() * 2 + \
        bl_analysis['p_sev_wt']/bl_analysis['p_sev_wt'].max() * 2 + \
        bl_analysis['count_bike']/bl_analysis['count_bike'].max() * 2 + \
        bl_analysis['b_sev_wt']/bl_analysis['b_sev_wt'].max() * 2

##### The easiest way to make your index is to change the weighting factor already in the code
0 is the same as deleting the factor
And... Keep creating if we have actually left enough time for you.

In [ ]:
bl_analysis['index4'] = \
        bl_analysis['coc.population.pct'] * 1 + \
        bl_analysis['elderly.population.pct'] * 1  + \
        bl_analysis['limited.english.households.pct'] * 1 + \
        bl_analysis['poverty.population.pct'] * 1 + \
        bl_analysis['single.parent.fam.pct'] * 1 + \
        bl_analysis['youth.population.pct'] * 1 + \
        bl_analysis['zero.vehicle.hh.pct'] * 1 + \
        bl_analysis['coc.pop.norm'] * 1  + \
        bl_analysis['elderly.pop.norm'] * 1 + \
        bl_analysis['limited.english.households.norm'] * 1 + \
        bl_analysis['poverty.population.norm'] * 1 + \
        bl_analysis['single.parent.fam.norm'] * 1 + \
        bl_analysis['youth.population.norm'] * 1 + \
        bl_analysis['zero.vehicle.hh.norm'] * 4 + \
        bl_analysis['count_ped']/bl_analysis['count_ped'].max() * 2 + \
        bl_analysis['p_sev_wt']/bl_analysis['p_sev_wt'].max() * 2 + \
        bl_analysis['count_bike']/bl_analysis['count_bike'].max() * 2 + \
        bl_analysis['b_sev_wt']/bl_analysis['b_sev_wt'].max() * 2

In [ ]:
bl_analysis['index5'] = \
        bl_analysis['coc.population.pct'] * 1 + \
        bl_analysis['elderly.population.pct'] * 1  + \
        bl_analysis['limited.english.households.pct'] * 1 + \
        bl_analysis['poverty.population.pct'] * 1 + \
        bl_analysis['single.parent.fam.pct'] * 1 + \
        bl_analysis['youth.population.pct'] * 1 + \
        bl_analysis['zero.vehicle.hh.pct'] * 1 + \
        bl_analysis['coc.pop.norm'] * 1  + \
        bl_analysis['elderly.pop.norm'] * 1 + \
        bl_analysis['limited.english.households.norm'] * 1 + \
        bl_analysis['poverty.population.norm'] * 1 + \
        bl_analysis['single.parent.fam.norm'] * 1 + \
        bl_analysis['youth.population.norm'] * 1 + \
        bl_analysis['zero.vehicle.hh.norm'] * 4 + \
        bl_analysis['count_ped']/bl_analysis['count_ped'].max() * 2 + \
        bl_analysis['p_sev_wt']/bl_analysis['p_sev_wt'].max() * 2 + \
        bl_analysis['count_bike']/bl_analysis['count_bike'].max() * 2 + \
        bl_analysis['b_sev_wt']/bl_analysis['b_sev_wt'].max() * 2

In [ ]:
for x in ['index1','index2','index3']:
    ax = bl_analysis.plot(figsize = (10, 10), column = x, cmap = "CMRmap_r",edgecolor = "lightgrey", linewidth = 0.5, legend = True)
    ax.set_axis_off()
    ax.set_title(x, fontsize=25)

### Final Action Item
#### Share screens/indices!

Does anyone want to share their index and their map?
